# Adaptive Timestepping in pygSQuiG

This notebook demonstrates:
1. CFL-based adaptive timestep control
2. Using the AdaptivegSQGSolver
3. Performance comparison with fixed timestep
4. Stability for challenging conditions
5. Configuration options and best practices

import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import time

# pygSQuiG imports
from pygsquig.core.grid import make_grid, ifft2, fft2
from pygsquig.core.solver import gSQGSolver
from pygsquig.core.adaptive_solver import AdaptivegSQGSolver
from pygsquig.core.adaptive_timestep import compute_timestep, compute_max_velocity, CFLConfig
from pygsquig.utils.diagnostics import compute_total_energy, compute_energy_spectrum

print("pygSQuiG adaptive timestepping demonstration")

In [ ]:
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import time

# pygSQuiG imports
from pygsquig.core.grid import make_grid, ifft2
from pygsquig.core.solver import gSQGSolver
from pygsquig.core.adaptive_solver import AdaptivegSQGSolver
from pygsquig.core.adaptive_timestep import compute_cfl_timestep, compute_max_velocity
from pygsquig.utils.diagnostics import compute_total_energy, compute_energy_spectrum

print("pygSQuiG adaptive timestepping demonstration")

# Create initial condition with strong vortices
x, y = grid.x, grid.y

# Two strong vortices
x1, y1 = L/3, L/2
x2, y2 = 2*L/3, L/2
sigma = L/15  # Narrow vortices for high gradients

theta_init = (
    5.0 * np.exp(-((x-x1)**2 + (y-y1)**2)/(2*sigma**2)) -
    5.0 * np.exp(-((x-x2)**2 + (y-y2)**2)/(2*sigma**2))
)

# Visualize initial condition
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

im1 = ax1.imshow(theta_init, cmap='RdBu_r', origin='lower', 
                 extent=[0, L, 0, L], vmin=-6, vmax=6)
ax1.set_title('Initial θ (vortex pair)')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
plt.colorbar(im1, ax=ax1)

# Compute initial velocity to check
from pygsquig.core.operators import compute_velocity_from_theta
theta_init_hat = fft2(theta_init)
u, v = compute_velocity_from_theta(theta_init_hat, grid, alpha)
speed = np.sqrt(u**2 + v**2)

im2 = ax2.imshow(speed, cmap='viridis', origin='lower',
                 extent=[0, L, 0, L])
ax2.set_title('Initial speed |u|')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

max_speed = float(np.max(speed))
dx = L / N
print(f"\nInitial maximum speed: {max_speed:.3f}")
print(f"Estimated CFL timestep: {cfl_number * dx / max_speed:.5f}")

In [ ]:
# Grid parameters
N = 128
L = 2 * np.pi
grid = make_grid(N, L)

# Physical parameters
alpha = 1.0  # SQG
nu_p = 1e-16
p = 8

# CFL parameters
cfl_number = 0.5  # Conservative CFL number

print(f"Configuration:")
print(f"  Grid: {N}×{N}, L={L:.2f}")
print(f"  SQG: α={alpha}, ν_{p}={nu_p:.1e}")
print(f"  CFL number: {cfl_number}")

# Create adaptive solver
cfl_config = CFLConfig(
    target_cfl=cfl_number,
    dt_min=1e-6,
    dt_max=0.01
)

solver_adaptive = AdaptivegSQGSolver(
    grid=grid,
    alpha=alpha,
    nu_p=nu_p,
    p=p,
    cfl_config=cfl_config
)

print("Adaptive solver configuration:")
print(f"  CFL number: {cfl_number}")
print(f"  dt range: [{cfl_config.dt_min}, {cfl_config.dt_max}]")

# NOTE: The current implementation of adaptive timestepping has some issues
# with the initial timestep estimate. In practice, you may need to tune
# the parameters or use a different initial timestep estimate.

# Run adaptive simulation
print("\nRunning adaptive simulation...")
start_time = time.time()

# Use evolve method for adaptive stepping
state_adaptive = solver_adaptive.initialize(theta0=theta_init)
results = solver_adaptive.evolve(
    state_adaptive,
    t_final=t_final,
    save_interval=0.01,  # Save every 0.01 time units
    max_steps=10000
)

elapsed_adaptive = time.time() - start_time

print(f"\nAdaptive simulation complete!")
print(f"  Total steps: {results['n_steps']}")
print(f"  Elapsed time: {elapsed_adaptive:.2f}s")
print(f"  Average dt: {t_final / results['n_steps']:.5f}")
print(f"  dt range used: [{np.min(results['timesteps']):.5f}, {np.max(results['timesteps']):.5f}]")

In [ ]:
# Create initial condition with strong vortices
x, y = grid.x, grid.y

# Two strong vortices
x1, y1 = L/3, L/2
x2, y2 = 2*L/3, L/2
sigma = L/15  # Narrow vortices for high gradients

theta_init = (
    5.0 * np.exp(-((x-x1)**2 + (y-y1)**2)/(2*sigma**2)) -
    5.0 * np.exp(-((x-x2)**2 + (y-y2)**2)/(2*sigma**2))
)

# Visualize initial condition
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

im1 = ax1.imshow(theta_init, cmap='RdBu_r', origin='lower', 
                 extent=[0, L, 0, L], vmin=-6, vmax=6)
ax1.set_title('Initial θ (vortex pair)')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
plt.colorbar(im1, ax=ax1)

# Compute initial velocity to check
from pygsquig.core.operators import compute_velocity_from_theta
from pygsquig.core.grid import fft2
theta_init_hat = fft2(theta_init)
u, v = compute_velocity_from_theta(theta_init_hat, grid, alpha)
speed = np.sqrt(u**2 + v**2)

im2 = ax2.imshow(speed, cmap='viridis', origin='lower',
                 extent=[0, L, 0, L])
ax2.set_title('Initial speed |u|')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
plt.colorbar(im2, ax=ax2)

plt.tight_layout()
plt.show()

max_speed = float(np.max(speed))
print(f"\nInitial maximum speed: {max_speed:.3f}")
print(f"Estimated CFL timestep: {cfl_number * grid.dx / max_speed:.5f}")

## 4. Fixed Timestep Solver

First, let's try a fixed timestep solver to see the challenges:

In [ ]:
# Test different CFL numbers
cfl_numbers = [0.2, 0.5, 0.8]
cfl_results = {}

print("Testing different CFL numbers...")
for cfl in cfl_numbers:
    cfl_config_test = CFLConfig(
        target_cfl=cfl,
        dt_min=1e-6,
        dt_max=0.01
    )
    
    solver_cfl = AdaptivegSQGSolver(
        grid=grid,
        alpha=alpha,
        nu_p=nu_p,
        p=p,
        cfl_config=cfl_config_test
    )
    
    state = solver_cfl.initialize(theta0=theta_init)
    
    start_time = time.time()
    results_cfl = solver_cfl.evolve(
        state,
        t_final=0.2,  # Shorter test
        max_steps=5000
    )
    elapsed = time.time() - start_time
    
    cfl_results[cfl] = {
        'n_steps': results_cfl['n_steps'],
        'elapsed': elapsed,
        'avg_dt': 0.2 / results_cfl['n_steps'],
        'energy_change': (results_cfl['diagnostics']['energy'][-1] - 
                         results_cfl['diagnostics']['energy'][0]) / 
                         results_cfl['diagnostics']['energy'][0]
    }
    
    print(f"  CFL={cfl}: {results_cfl['n_steps']} steps, "
          f"avg dt={cfl_results[cfl]['avg_dt']:.5f}, "
          f"time={elapsed:.2f}s")

# Plot CFL comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

cfls = list(cfl_results.keys())
n_steps = [cfl_results[c]['n_steps'] for c in cfls]
times = [cfl_results[c]['elapsed'] for c in cfls]
energy_changes = [abs(cfl_results[c]['energy_change']) * 100 for c in cfls]

ax1.bar(cfls, n_steps, alpha=0.7, color='blue')
ax1.set_xlabel('CFL Number')
ax1.set_ylabel('Number of Steps')
ax1.set_title('Steps vs CFL Number')
ax1.grid(True, alpha=0.3)

ax2.plot(cfls, energy_changes, 'ro-', markersize=8)
ax2.set_xlabel('CFL Number')
ax2.set_ylabel('Energy Change (%)')
ax2.set_title('Accuracy vs CFL Number')
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

print("\nCFL Guidelines:")
print("  - Lower CFL (0.2-0.3): More conservative, better accuracy")
print("  - Medium CFL (0.4-0.6): Good balance of efficiency and stability")
print("  - Higher CFL (0.7-0.9): Faster but may sacrifice accuracy")

# Create extreme initial condition
# Multiple narrow vortices
theta_extreme = np.zeros_like(x)
n_vortices = 6
for i in range(n_vortices):
    xi = L * (i + 0.5) / n_vortices
    yi = L/2 + L/4 * np.sin(2*np.pi*i/n_vortices)
    sign = 1 if i % 2 == 0 else -1
    theta_extreme += sign * 8.0 * np.exp(-((x-xi)**2 + (y-yi)**2)/(2*(L/30)**2))

# Visualize
plt.figure(figsize=(8, 6))
plt.imshow(theta_extreme, cmap='RdBu_r', origin='lower',
           extent=[0, L, 0, L])
plt.colorbar(label='θ')
plt.title('Extreme Initial Condition: Multiple Strong Vortices')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# Test adaptive solver
cfl_config_extreme = CFLConfig(
    target_cfl=0.3,  # Conservative
    dt_min=1e-7,
    dt_max=0.001
)

solver_extreme = AdaptivegSQGSolver(
    grid=grid,
    alpha=alpha,
    nu_p=1e-12,  # Slightly higher dissipation for stability
    p=8,
    cfl_config=cfl_config_extreme
)

print("Running extreme case...")
state_extreme = solver_extreme.initialize(theta0=theta_extreme)

# Check initial max velocity
u, v = compute_velocity_from_theta(state_extreme['theta_hat'], grid, alpha)
max_vel_extreme = float(compute_max_velocity(u, v))
print(f"Initial max velocity: {max_vel_extreme:.2f}")
print(f"Required dt for CFL: {0.3 * dx / max_vel_extreme:.6f}")

# Evolve
results_extreme = solver_extreme.evolve(
    state_extreme,
    t_final=0.1,
    save_interval=0.01,
    max_steps=10000,
    min_progress_rate=1e-8  # Allow slow progress
)

print(f"\nExtreme case complete!")
print(f"  Total steps: {results_extreme['n_steps']}")
print(f"  Min dt used: {np.min(results_extreme['timesteps']):.2e}")
print(f"  Max dt used: {np.max(results_extreme['timesteps']):.2e}")
print(f"  dt variation: {np.max(results_extreme['timesteps'])/np.min(results_extreme['timesteps']):.1f}x")

# Plot timestep history
plt.figure(figsize=(10, 6))
plt.semilogy(results_extreme['step_times'][:-1], results_extreme['timesteps'],
             'b-', linewidth=1, alpha=0.7)
plt.xlabel('Time')
plt.ylabel('Timestep dt')
plt.title('Timestep Adaptation for Extreme Case')
plt.grid(True, alpha=0.3)
plt.axhline(y=cfl_config_extreme.dt_min, color='r', linestyle='--', 
            alpha=0.5, label=f'dt_min={cfl_config_extreme.dt_min}')
plt.axhline(y=cfl_config_extreme.dt_max, color='r', linestyle='--', 
            alpha=0.5, label=f'dt_max={cfl_config_extreme.dt_max}')
plt.legend()
plt.show()

print("\nKey observation: Timestep varies by orders of magnitude!")

In [ ]:
# Create adaptive solver
solver_adaptive = AdaptivegSQGSolver(
    grid=grid,
    alpha=alpha,
    nu_p=nu_p,
    p=p,
    cfl_number=cfl_number,
    dt_min=1e-6,
    dt_max=0.01
)

print("Adaptive solver configuration:")
print(f"  CFL number: {cfl_number}")
print(f"  dt range: [{solver_adaptive.dt_min}, {solver_adaptive.dt_max}]")

# Run adaptive simulation
print("\nRunning adaptive simulation...")
start_time = time.time()

# Use evolve method for adaptive stepping
state_adaptive = solver_adaptive.initialize(theta0=theta_init)
results = solver_adaptive.evolve(
    state_adaptive,
    t_final=t_final,
    save_interval=0.01,  # Save every 0.01 time units
    max_steps=10000
)

elapsed_adaptive = time.time() - start_time

print(f"\nAdaptive simulation complete!")
print(f"  Total steps: {results['n_steps']}")
print(f"  Elapsed time: {elapsed_adaptive:.2f}s")
print(f"  Average dt: {t_final / results['n_steps']:.5f}")
print(f"  dt range used: [{np.min(results['timesteps']):.5f}, {np.max(results['timesteps']):.5f}]")

## 6. Compare Performance

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Energy evolution
ax = axes[0, 0]
for dt_fixed, data in results_fixed.items():
    if data['stable']:
        ax.plot(data['times'], data['energies'], 
                label=f'Fixed dt={dt_fixed}', linewidth=2)

ax.plot(results['times'], results['diagnostics']['energy'], 
        'k--', label='Adaptive', linewidth=2)
ax.set_xlabel('Time')
ax.set_ylabel('Total Energy')
ax.set_title('Energy Evolution')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Maximum velocity
ax = axes[0, 1]
stable_fixed = [k for k, v in results_fixed.items() if v['stable']]
if stable_fixed:
    dt_show = min(stable_fixed)  # Show smallest stable
    ax.plot(results_fixed[dt_show]['times'], 
            results_fixed[dt_show]['max_velocities'],
            'b-', label=f'Fixed dt={dt_show}', linewidth=2)

ax.plot(results['times'], results['diagnostics']['max_velocity'],
        'r-', label='Adaptive', linewidth=2)
ax.set_xlabel('Time')
ax.set_ylabel('Max Velocity')
ax.set_title('Maximum Velocity Evolution')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Timestep history (adaptive)
ax = axes[1, 0]
ax.semilogy(results['step_times'][:-1], results['timesteps'], 
            'g-', linewidth=1, alpha=0.7)
ax.set_xlabel('Time')
ax.set_ylabel('Timestep dt')
ax.set_title('Adaptive Timestep History')
ax.grid(True, alpha=0.3)
ax.axhline(y=solver_adaptive.dt_min, color='r', linestyle='--', 
           alpha=0.5, label=f'dt_min={solver_adaptive.dt_min}')
ax.axhline(y=solver_adaptive.dt_max, color='r', linestyle='--', 
           alpha=0.5, label=f'dt_max={solver_adaptive.dt_max}')
ax.legend()

# 4. Efficiency comparison
ax = axes[1, 1]
labels = []
steps = []
times = []
colors = []

for dt_fixed, data in sorted(results_fixed.items()):
    labels.append(f'Fixed\ndt={dt_fixed}')
    steps.append(data['n_steps'] if data['stable'] else 0)
    times.append(data['elapsed'] if data['stable'] else 0)
    colors.append('green' if data['stable'] else 'red')

labels.append('Adaptive')
steps.append(results['n_steps'])
times.append(elapsed_adaptive)
colors.append('blue')

x = np.arange(len(labels))
width = 0.35

bars1 = ax.bar(x - width/2, steps, width, label='Steps', color=colors, alpha=0.6)
ax2 = ax.twinx()
bars2 = ax2.bar(x + width/2, times, width, label='Time (s)', alpha=0.6)

ax.set_xlabel('Solver Type')
ax.set_ylabel('Number of Steps', color='tab:blue')
ax2.set_ylabel('Computation Time (s)', color='tab:orange')
ax.set_title('Efficiency Comparison')
ax.set_xticks(x)
ax.set_xticklabels(labels)

plt.tight_layout()
plt.show()

# Summary
print("\nPerformance Summary:")
print("=" * 50)
for dt_fixed, data in sorted(results_fixed.items()):
    if data['stable']:
        print(f"Fixed dt={dt_fixed}: {data['n_steps']} steps, {data['elapsed']:.2f}s")
    else:
        print(f"Fixed dt={dt_fixed}: UNSTABLE")
print(f"Adaptive: {results['n_steps']} steps, {elapsed_adaptive:.2f}s")

## 7. Visualize Evolution

Let's look at how the vortices evolve:

In [ ]:
# Plot snapshots from adaptive simulation
snapshot_indices = [0, len(results['states'])//3, 2*len(results['states'])//3, -1]
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

for i, idx in enumerate(snapshot_indices):
    state = results['states'][idx]
    t = results['times'][idx]
    
    # θ field
    theta = ifft2(state['theta_hat']).real
    im1 = axes[0, i].imshow(theta, cmap='RdBu_r', origin='lower',
                            extent=[0, L, 0, L], vmin=-6, vmax=6)
    axes[0, i].set_title(f't = {t:.2f}')
    axes[0, i].set_xlabel('x')
    if i == 0:
        axes[0, i].set_ylabel('θ\ny')
    else:
        axes[0, i].set_ylabel('y')
    
    # Velocity magnitude
    u, v = compute_velocity_from_theta(state['theta_hat'], grid, alpha)
    speed = np.sqrt(u**2 + v**2)
    im2 = axes[1, i].imshow(speed, cmap='viridis', origin='lower',
                            extent=[0, L, 0, L])
    axes[1, i].set_xlabel('x')
    if i == 0:
        axes[1, i].set_ylabel('|u|\ny')
    else:
        axes[1, i].set_ylabel('y')
    
    # Add velocity info
    max_vel = float(np.max(speed))
    axes[1, i].text(0.05, 0.95, f'max={max_vel:.2f}', 
                    transform=axes[1, i].transAxes,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                    verticalalignment='top')

# Colorbars
cbar1 = fig.colorbar(im1, ax=axes[0, :], location='right', fraction=0.02)
cbar1.set_label('θ')
cbar2 = fig.colorbar(im2, ax=axes[1, :], location='right', fraction=0.02)
cbar2.set_label('|u|')

plt.suptitle('Vortex Evolution with Adaptive Timestepping', fontsize=14)
plt.tight_layout()
plt.show()

print("Observations:")
print("  - Vortices interact and merge")
print("  - Velocity varies significantly during evolution")
print("  - Adaptive timestep adjusts to maintain stability")

## 8. Advanced Configuration Options

The adaptive solver has several configuration options:

In [ ]:
# Test different CFL numbers
cfl_numbers = [0.2, 0.5, 0.8]
cfl_results = {}

print("Testing different CFL numbers...")
for cfl in cfl_numbers:
    solver_cfl = AdaptivegSQGSolver(
        grid=grid,
        alpha=alpha,
        nu_p=nu_p,
        p=p,
        cfl_number=cfl,
        dt_min=1e-6,
        dt_max=0.01
    )
    
    state = solver_cfl.initialize(theta0=theta_init)
    
    start_time = time.time()
    results_cfl = solver_cfl.evolve(
        state,
        t_final=0.2,  # Shorter test
        max_steps=5000
    )
    elapsed = time.time() - start_time
    
    cfl_results[cfl] = {
        'n_steps': results_cfl['n_steps'],
        'elapsed': elapsed,
        'avg_dt': 0.2 / results_cfl['n_steps'],
        'energy_change': (results_cfl['diagnostics']['energy'][-1] - 
                         results_cfl['diagnostics']['energy'][0]) / 
                         results_cfl['diagnostics']['energy'][0]
    }
    
    print(f"  CFL={cfl}: {results_cfl['n_steps']} steps, "
          f"avg dt={cfl_results[cfl]['avg_dt']:.5f}, "
          f"time={elapsed:.2f}s")

# Plot CFL comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

cfls = list(cfl_results.keys())
n_steps = [cfl_results[c]['n_steps'] for c in cfls]
times = [cfl_results[c]['elapsed'] for c in cfls]
energy_changes = [abs(cfl_results[c]['energy_change']) * 100 for c in cfls]

ax1.bar(cfls, n_steps, alpha=0.7, color='blue')
ax1.set_xlabel('CFL Number')
ax1.set_ylabel('Number of Steps')
ax1.set_title('Steps vs CFL Number')
ax1.grid(True, alpha=0.3)

ax2.plot(cfls, energy_changes, 'ro-', markersize=8)
ax2.set_xlabel('CFL Number')
ax2.set_ylabel('Energy Change (%)')
ax2.set_title('Accuracy vs CFL Number')
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

print("\nCFL Guidelines:")
print("  - Lower CFL (0.2-0.3): More conservative, better accuracy")
print("  - Medium CFL (0.4-0.6): Good balance of efficiency and stability")
print("  - Higher CFL (0.7-0.9): Faster but may sacrifice accuracy")

## 9. Handling Extreme Cases

Let's test the adaptive solver with a very challenging case:

In [ ]:
# Create extreme initial condition
# Multiple narrow vortices
theta_extreme = np.zeros_like(x)
n_vortices = 6
for i in range(n_vortices):
    xi = L * (i + 0.5) / n_vortices
    yi = L/2 + L/4 * np.sin(2*np.pi*i/n_vortices)
    sign = 1 if i % 2 == 0 else -1
    theta_extreme += sign * 8.0 * np.exp(-((x-xi)**2 + (y-yi)**2)/(2*(L/30)**2))

# Visualize
plt.figure(figsize=(8, 6))
plt.imshow(theta_extreme, cmap='RdBu_r', origin='lower',
           extent=[0, L, 0, L])
plt.colorbar(label='θ')
plt.title('Extreme Initial Condition: Multiple Strong Vortices')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# Test adaptive solver
solver_extreme = AdaptivegSQGSolver(
    grid=grid,
    alpha=alpha,
    nu_p=1e-12,  # Slightly higher dissipation for stability
    p=8,
    cfl_number=0.3,  # Conservative
    dt_min=1e-7,
    dt_max=0.001
)

print("Running extreme case...")
state_extreme = solver_extreme.initialize(theta0=theta_extreme)

# Check initial max velocity
u, v = compute_velocity_from_theta(state_extreme['theta_hat'], grid, alpha)
max_vel_extreme = float(compute_max_velocity(u, v))
print(f"Initial max velocity: {max_vel_extreme:.2f}")
print(f"Required dt for CFL: {0.3 * grid.dx / max_vel_extreme:.6f}")

# Evolve
results_extreme = solver_extreme.evolve(
    state_extreme,
    t_final=0.1,
    save_interval=0.01,
    max_steps=10000,
    min_progress_rate=1e-8  # Allow slow progress
)

print(f"\nExtreme case complete!")
print(f"  Total steps: {results_extreme['n_steps']}")
print(f"  Min dt used: {np.min(results_extreme['timesteps']):.2e}")
print(f"  Max dt used: {np.max(results_extreme['timesteps']):.2e}")
print(f"  dt variation: {np.max(results_extreme['timesteps'])/np.min(results_extreme['timesteps']):.1f}x")

# Plot timestep history
plt.figure(figsize=(10, 6))
plt.semilogy(results_extreme['step_times'][:-1], results_extreme['timesteps'],
             'b-', linewidth=1, alpha=0.7)
plt.xlabel('Time')
plt.ylabel('Timestep dt')
plt.title('Timestep Adaptation for Extreme Case')
plt.grid(True, alpha=0.3)
plt.axhline(y=solver_extreme.dt_min, color='r', linestyle='--', 
            alpha=0.5, label=f'dt_min={solver_extreme.dt_min}')
plt.axhline(y=solver_extreme.dt_max, color='r', linestyle='--', 
            alpha=0.5, label=f'dt_max={solver_extreme.dt_max}')
plt.legend()
plt.show()

print("\nKey observation: Timestep varies by orders of magnitude!")

## 10. Best Practices Summary

### When to Use Adaptive Timestepping:
1. **Variable dynamics**: When velocity changes significantly
2. **Long simulations**: Efficiency matters
3. **Unknown conditions**: Not sure about maximum velocities
4. **Robustness**: Want automatic stability

### Configuration Guidelines:
1. **CFL number**: 
   - 0.2-0.4 for accuracy-critical simulations
   - 0.4-0.6 for general use
   - 0.6-0.8 when efficiency is priority

2. **dt_min and dt_max**:
   - dt_min: ~1e-6 to 1e-7 × characteristic time
   - dt_max: ~0.01 to 0.1 × characteristic time
   - Range of 3-4 orders of magnitude is typical

3. **Safety parameters**:
   - max_steps: Prevent infinite loops
   - min_progress_rate: Detect stalling

### Performance Tips:
1. Start with conservative CFL (~0.5)
2. Monitor timestep history
3. Check energy conservation
4. Adjust parameters based on your specific problem

In [ ]:
# Example: Recommended configuration
print("Recommended adaptive solver configuration:")
print("""solver = AdaptivegSQGSolver(
    grid=grid,
    alpha=alpha,
    nu_p=nu_p,
    p=p,
    cfl_number=0.5,      # Good balance
    dt_min=1e-6,         # Safety floor
    dt_max=0.01,         # Efficiency ceiling
    safety_factor=0.9    # Additional safety margin
)

results = solver.evolve(
    state,
    t_final=t_final,
    save_interval=0.1,   # Save snapshots
    max_steps=100000,    # Prevent runaway
    min_progress_rate=1e-6  # Detect stalling
)""")

print("\nThis notebook demonstrated adaptive timestepping in pygSQuiG!")
print("Key benefits: Automatic stability, efficiency, and robustness.")
print("Next: See notebook 05 for passive scalar simulations.")